In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pandas as pd
import numpy as np
import copy
# import torch
from torch.utils.data import Dataset
# from torchvision import datasets
import torchvision
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torchvision.io import read_image
import torchvision.models as models
# from torch.autograd import Variable

In [2]:
!pip install -q efficientnet_pytorch

In [6]:
# pretrained models
from efficientnet_pytorch import EfficientNet
from PIL import Image
from torchvision import transforms

In [8]:
img_dir = '/kaggle/input/col774-2022/images/images/'
train_path_x ='/kaggle/input/col774-2022/train_x.csv'
train_path_y ='/kaggle/input/col774-2022/train_y.csv'
test_path_x = '/kaggle/input/col774-2022/non_comp_test_x.csv'
test_path_y = '/kaggle/input/col774-2022/non_comp_test_y.csv'
final_test_path_x='/kaggle/input/col774-2022/comp_test_x.csv'


In [9]:
class BookCoverImageDataset(Dataset):
    def __init__(self, train_path_x, train_path_y, img_dir,transform=None):
        self.img_labels = pd.read_csv(train_path_y)
        self.img_names  = pd.read_csv(train_path_x)
        self.img_names  = self.img_names.set_index('Id')
        self.img_labels = self.img_labels.set_index('Id')
        self.img_dir = img_dir
        self.xtransform=transform
        # self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])    

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):        
        img_path = os.path.join(self.img_dir, self.img_names['Cover_image_name'][idx])
        image = Image.open(img_path)
#         image = image/255.0
        # image = self.normalize(image)
        image =self.xtransform(image)
        label = self.img_labels['Genre'][idx]
        # features = feature_extractor(image)
        label=torch.tensor(label)
        return image, label

In [10]:
# Preprocess image
model_name = 'efficientnet-b3'
image_size = EfficientNet.get_image_size(model_name) # 300
tfms = transforms.Compose([transforms.Resize(image_size), transforms.CenterCrop(image_size), 
                           transforms.ToTensor(),
                           transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
# img = tfms(img).unsqueeze(0)

In [11]:
training_data = BookCoverImageDataset(train_path_x, train_path_y, img_dir,tfms)
test_data = BookCoverImageDataset(test_path_x, test_path_y, img_dir,tfms)
train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
test_dataloader  = DataLoader(test_data, batch_size=32, shuffle=True)

In [12]:
model = EfficientNet.from_pretrained('efficientnet-b3')

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth


  0%|          | 0.00/47.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b3


In [18]:
num_classes = 30
model._fc=nn.Linear(in_features=1536, out_features=num_classes, bias=True)
model._fc

In [22]:
fine_tune = True
for params in model.parameters():
    params.requires_grad = True
num_classes = 30

In [23]:
def accuracy(model, dataloader):
    softmax = nn.Softmax(dim=1)
    model.eval()
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')    
    correct = 0.0
    with torch.no_grad():    
        for x,y in dataloader:
            train_x, train_y = x, y#batch[0], batch[1]
            train_x = train_x.to(device)
            train_y = train_y.to(device)
            logits = model(train_x)        
#             _, preds = torch.max(output.data, 1)
            preds=softmax(logits)
            correct += (preds.argmax(1) == train_y).type(torch.float).sum().item()
    return (correct/len(dataloader.dataset))*100

In [27]:
def train(model, train_dataloader, val_dataloader, optimizer, loss_fn, epochs=10, verbose=True, checkpoint=1):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')    
    model.to(device)
    loss_fn.to(device)
    model.train()
    best_model, best_score = None, 0.0

    loss_vals, val_accuracy, train_accuracy = [], [], []
    for epoch in range(epochs):    
        correct = 0.0
        model.train()    
        epoch_loss = 0.0
        cnt=0
        for batch,(x,y) in enumerate(train_dataloader):                        
            train_x, train_y = x,y#batch[0], batch[1]
            train_x = train_x.to(device)
            train_y = train_y.to(device)
            #fprop
            outputs = model(train_x)
            loss = loss_fn(outputs, train_y)
            optimizer.zero_grad()
            #bprop
            loss.backward()
            #update params
            optimizer.step()
            cnt+=1
#             _, preds = torch.max(outputs.data, 1)
#             correct += (preds == train_y).sum().item()
            epoch_loss += loss.item()  
#             print(loss.item())
        
#         train_acc = correct/len(train_dataloader.dataset)
        
        loss_vals.append(epoch_loss/cnt)
        val_accuracy.append(accuracy(model, test_dataloader))
        train_accuracy.append(accuracy(model, train_dataloader))

        if best_score < val_accuracy[-1]:
            best_score = val_accuracy[-1]
            best_model = copy.deepcopy(model)            

        if verbose and epoch%checkpoint == 0:                 
            print('epoch = {}'.format(epoch))       
            print('val_accuracy = {}'.format(val_accuracy[-1]))
            print('train_accuracy = {}'.format(train_accuracy[-1]))                        
            print('epoch loss = {}'.format(loss_vals[-1]))
    return model,train_accuracy, val_accuracy, loss_vals

In [25]:
from time import time

In [ ]:
lr = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()
t1=time()
model,train_accuracy, val_accuracy, loss = train(model, train_dataloader, test_dataloader, optimizer, loss_fn, epochs=5)
t=time()-t1
print('time',t)

epoch = 0
val_accuracy = 22.54385964912281
train_accuracy = 28.596491228070175
epoch loss = 2.5876652634623345
epoch = 1
val_accuracy = 26.0
train_accuracy = 35.45906432748538
epoch loss = 2.420012412169361


epoch = 0
val_accuracy = 21.263157894736842
train_accuracy = 24.01169590643275
epoch loss = 2.8328071044247434
time 1839.5948495864868

In [ ]:
# save the model
save_path = '/kaggle/working/model_cnn.pth'
torch.save(model.state_dict(), save_path)